In [11]:
import Metashape
import geopandas as gpd
import os
from shapely.geometry import Polygon

project_fld = "/data/ATM/data_1/sfm/agi_projects/CA1848/"
project_path = os.path.join(project_fld, "CA1848.psx")
save_path = os.path.join(project_fld, "shapes", "footprints.shp")
doc = Metashape.Document()
doc.open(project_path)
chunk = doc.chunks[0]
# Collect camera footprints
footprints = []

T = chunk.transform.matrix

chunk_crs = chunk.crs.geoccs
if chunk_crs is None:
    chunk_crs = Metashape.CoordinateSystem('LOCAL')

if chunk.elevation:
    surface = chunk.elevation
elif chunk.model:
    surface = chunk.model
elif chunk.point_cloud:
    surface = chunk.point_cloud
else:
    surface = chunk.tie_points

for camera in chunk.cameras:
    if not camera.transform:
        continue  # Skip cameras without valid transformation

    sensor = camera.sensor
    corners = list()
    for (x, y) in [[0, 0], [sensor.width - 1, 0], [sensor.width - 1, sensor.height - 1], [0, sensor.height - 1]]:
        ray_origin = camera.unproject(Metashape.Vector([x, y, 0]))
        ray_target = camera.unproject(Metashape.Vector([x, y, 1]))
        if type(surface) == Metashape.Elevation:
            dem_origin = T.mulp(ray_origin)
            dem_target = T.mulp(ray_target)
            dem_origin = Metashape.OrthoProjection.transform(dem_origin, chunk_crs, surface.projection)
            dem_target = Metashape.OrthoProjection.transform(dem_target, chunk_crs, surface.projection)
            corner = surface.pickPoint(dem_origin, dem_target)
            if corner:
                corner = Metashape.OrthoProjection.transform(corner, surface.projection, chunk_crs)
                corner = T.inv().mulp(corner)
        else:
            corner = surface.pickPoint(ray_origin, ray_target)
        if not corner:
            corner = chunk.tie_points.pickPoint(ray_origin, ray_target)
        if not corner:
            break
        corner = chunk.crs.project(T.mulp(corner))
        corners.append(corner)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=footprints, crs="EPSG:3031")  # Adjust CRS as necessary

# Save to shapefile
gdf.to_file(save_path, driver='ESRI Shapefile')

LoadProject: path = /data/ATM/data_1/sfm/agi_projects/CA1848/CA1848.psx
loaded project in 0.000857 sec


/home/fdahle/miniconda3/envs/tma_env/lib/python3.11/site-packages/geopandas/io/file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
